In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import datetime

In [2]:
data_path = '../data/'
filename1 = 'sdwpf_baidukddcup2022_full.CSV'
df = pd.read_csv(data_path + filename1)
df.dropna(axis=0)

,TurbID,Day,Tmstamp,Wspd,Wdir,Etmp,Itmp,Ndir,Pab1,Pab2,Pab3,Prtv,Patv
0,1,1,00:00,12.23,-0.83,29.08,41.90,-23.73,1.07,1.07,1.07,-0.21,1549.53
1,1,1,00:10,11.58,-3.32,29.01,42.01,-23.70,1.06,1.06,1.06,-0.25,1549.71
2,1,1,00:20,11.21,-1.38,29.17,42.24,-28.84,1.04,1.04,1.04,-0.25,1534.77
3,1,1,00:30,10.84,0.06,29.46,42.43,-31.39,1.03,1.03,1.03,-0.25,1508.20
4,1,1,00:40,11.03,2.03,29.82,42.77,-31.39,1.03,1.03,1.03,-66.01,1517.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3550459,134,184,23:10,2.36,-74.19,7.30,11.70,238.59,90.39,90.37,90.36,-0.30,-0.30
3550460,134,184,23:20,1.72,-67.92,7.21,11.70,238.59,90.39,90.37,90.36,-0.30,-0.30
3550461,134,184,23:30,1.46,-59.15,7.10,11.70,238.59,90.39,90.37,90.36,-0.30,-0.30
3550462,134,184,23:40,1.31,-64.11,7.10,11.70,238.59,90.39,90.37,90.36,-0.30,-0.30


In [3]:
i_tmp = df['Itmp'].values
mean = np.nanmean(i_tmp)
std = np.nanstd(i_tmp)
threshold_low = mean - 3 * std
threshold_high = mean + 3 * std
i_tmp[i_tmp > threshold_high] = np.nan
i_tmp[i_tmp < threshold_low] = np.nan
mean = np.nanmean(i_tmp)
std = np.nanstd(i_tmp)
threshold_low = mean - 3 * std
threshold_high = mean + 3 * std

In [3]:
e_tmp = df['Etmp'].values

In [4]:
mean = np.nanmean(e_tmp)
std = np.nanstd(e_tmp)
threshold_low = mean - 3 * std
threshold_high = mean + 3 * std
#
# valid_index = 288
# normal = (e_tmp > threshold_low) & (e_tmp < threshold_high) & ~np.isnan(e_tmp)
# print(normal[valid_index - 288: valid_index].all())
# while not normal[valid_index - 288: valid_index].all():
#     exist = np.argwhere(normal[valid_index + 1:] == True)
#     valid_index = exist[0][0] + valid_index + 1
#     print(exist[0][0] + valid_index + 1)

In [5]:
e_tmp[e_tmp > threshold_high] = np.nan
e_tmp[e_tmp < threshold_low] = np.nan

In [6]:
mean = np.nanmean(e_tmp)
std = np.nanstd(e_tmp)
threshold_low = mean - 3 * std
threshold_high = mean + 3 * std

In [7]:
# fig = plt.figure(figsize=(20, 6), dpi=100)
# x_major_locator = MultipleLocator(1)
# ax = fig.gca()
# ax.xaxis.set_major_locator(x_major_locator)
# x_axis = np.array(range(len(e_tmp))) / 144
# th_low_point = np.empty(len(e_tmp))
# th_low_point.fill(threshold_low)
# th_high_point = np.empty(len(e_tmp))
# th_high_point.fill(threshold_high)
# plt.plot(x_axis[:2000], e_tmp[:2000])
# plt.plot(x_axis[:2000], th_low_point[:2000], color='black')
# plt.plot(x_axis[:2000], th_high_point[:2000], color='black')
# plt.legend()
# fig.show()

In [8]:
# normal = (e_tmp > threshold_low) & (e_tmp < threshold_high) & ~np.isnan(e_tmp)

In [9]:
# %%time
# valid_index = 1481
# print(normal[valid_index - 288: valid_index].all())
# if normal[valid_index - 288: valid_index].all():
#     print(valid_index)
# while not normal[valid_index - 288: valid_index].all():
#     valid_index += 1
#     print(valid_index)

In [10]:
from notebooks.tmp_dataset import TMPDataset
from notebooks.tmp_train import train

In [11]:
batch_size = 32
model_name = 'tmp_lstm'
cur_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "./logs/" + cur_time + '-' + model_name
model_path = './tmp_model/' + cur_time + '-' + model_name
dataset = TMPDataset(e_tmp)
train_ls = train(log_dir, model_path, dataset, batch_size=batch_size, sample_size=0.01)

Shape of test X:  torch.Size([32, 288, 1])
Shape of test y:  torch.Size([32, 1]) torch.float32
Layer (type:depth-idx)                   Input Shape               Kernel Shape              Output Shape
LSTMModel                                --                        --                        --
├─LSTM: 1-1                              [32, 288, 1]              --                        [32, 288, 32]
│    └─weight_ih_l0                                                [64, 1]
│    └─weight_hh_l0                                                [64, 16]
│    └─bias_ih_l0                                                  [64]
│    └─bias_hh_l0                                                  [64]
│    └─weight_ih_l0_reverse                                        [64, 1]
│    └─weight_hh_l0_reverse                                        [64, 16]
│    └─bias_ih_l0_reverse                                          [64]
│    └─bias_hh_l0_reverse                                          [64]
│    └

KeyboardInterrupt: 